In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
from aerb_unv_reader import UNV

In [3]:
from material_prop_reader import material_section_read

In [4]:
with open('EPR_mat.pickle', 'rb') as handle:
    EPR_mat = pickle.load(handle)
with open('GROUPS.pickle', 'rb') as handle:
    GROUPS = pickle.load(handle)
with open('ELEM_GROUPS.pickle', 'rb') as handle:
    ELEM_GROUPS = pickle.load(handle)
with open('NODE_GROUPS.pickle', 'rb') as handle:
    NODE_GROUPS = pickle.load(handle)
with open('MATERIALS.pickle', 'rb') as handle:
    MATERIALS = pickle.load(handle)
with open('ELEMENT_MATERIAL_INX.pickle', 'rb') as handle:
    ELEMENT_MATERIAL_INX =  pickle.load(handle)
with open('Unique_MATERIALS.pickle', 'rb') as handle:
    Unique_MATERIALS = pickle.load(handle)

In [5]:
EPR_geo = UNV('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/CRUAS_NPP.unv')

# Truss Elements

In [6]:
ROD_ELEMENTS = []
for key,value in EPR_mat.section_details['ROD'].items():
    ROD_ELEMENTS = ROD_ELEMENTS + list(ELEM_GROUPS[key])

In [7]:
## READ ALL ELEMENTS
ELEMENTS = {}
for key,value in EPR_geo.fem['elements'][21].items():
    ELEMENTS[key] = {'type':21, 'connectivity':value[-2:]}

In [8]:
TRUSS_Sections = {}
TRUSS = {}
MASTERLIST = {}
for gid, details in EPR_mat.section_details['ROD'].items():
    if details['type'] == 'RECTANGLE':
        area  =  details['parameters'][0]* details['parameters'][0]
    elif details['type'] == 'CIRCLE':
        area  =  np.pi*details['parameters'][0]**2
    elif details['type'] == 'GENERAL':
        area  =  details['parameters']
    for eid in ELEM_GROUPS[gid]:
        if eid in MASTERLIST.keys():
            print(eid,area,MASTERLIST[eid])
        else:
            MASTERLIST[eid] = area
        if area in TRUSS.keys():
            TRUSS[area].append(eid)
        else:
             TRUSS[area] = [eid]
    """
    
        Spring[eid] = {'nodes':ELEMENTS[eid]['connectivity'],\
                      'DOFs':[1,2,3], 'KK':details['parameters']}"""
for key,els in TRUSS.items():
    TRUSS[key] = sorted(set(els))

11708 1.0 1.0
11709 1.0 1.0
11710 1.0 1.0
11711 1.0 1.0
12843 1.0 1.0
12844 1.0 1.0
12845 1.0 1.0
12846 1.0 1.0


In [19]:
with open('TRUSS.pickle', 'wb') as handle:
    pickle.dump(TRUSS, handle, protocol=1)
with open('TRUSS_ELIST.pickle', 'wb') as handle:
    pickle.dump(list(MASTERLIST.keys()), handle, protocol=1)

In [20]:
for key,els in TRUSS.items():
    print(key, len(els))

0.25 6
0.48999999999999994 1
0.029559245277626364 7
0.08244795760081053 21
1.0 626


In [21]:
# ALL TRUSS INFO ARE GOING INTO ABAQUS - CONFIRMED